In [1]:
import numpy as np
import pymysql
import matplotlib.pyplot as plt
import random
import time
import math
from sklearn.linear_model import LogisticRegression

num_train=260864
num_test=261477

In [2]:
conn = pymysql.connect(host='localhost', port=3306,user='root',passwd='root',db='cse258',charset='UTF8')
cur = conn.cursor()

In [3]:
def calc_user_feature(user):
    cmd='SELECT seller_id,COUNT(*) FROM user_log WHERE user_id='+user+' AND action_type=2 GROUP BY seller_id'
    cur.execute(cmd)
    result=cur.fetchall()
    repeat_seller=0
    for r in result:
        if r[1]>1:
            repeat_seller+=1
    return (len(result),repeat_seller)

def calc_seller_feature(seller):
    cmd='SELECT user_id,COUNT(*) FROM user_log WHERE seller_id='+seller+' AND action_type=2 GROUP BY user_id'
    cur.execute(cmd)
    result=cur.fetchall()
    repeat_user=0
    for r in result:
        if r[1]>1:
            repeat_user+=1
    return (len(result),repeat_user)

In [4]:
result_user={}
result_seller={}

X=[]
f_train=open('data_format1/train_format1.csv')
cnt=0
print_every=5000
for line in f_train:
    l=line.strip()
    if l=='user_id,merchant_id,label':
        continue
    [user,seller,this_y]=l.split(',')
    
    if user not in result_user:
        l1,l2=calc_user_feature(user)
        result_user[user]=(l1,l2)
    else:
        l1,l2=result_user[user]
        
#     if seller not in result_seller:
#         l3,l4=calc_seller_feature(seller)
#         result_seller[seller]=(l3,l4)
#     else:
#         l3,l4=result_seller[seller]
        
    X.append([l1,l2])
    
    if cnt==0 or (cnt+1)%print_every==0 or (cnt+1)==num_train:
        print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())),
              'finished '+str(cnt+1)+' pairs')
    cnt+=1

X=np.array(X)
print(X.shape)
np.save('data/x_bias.npy', X)

2017-11-28 18:39:53 finished 1 pairs
2017-11-28 18:42:40 finished 5000 pairs
2017-11-28 18:47:00 finished 10000 pairs
2017-11-28 18:52:51 finished 15000 pairs
2017-11-28 18:59:16 finished 20000 pairs
2017-11-28 19:03:57 finished 25000 pairs
2017-11-28 19:08:19 finished 30000 pairs
2017-11-28 19:12:25 finished 35000 pairs
2017-11-28 19:17:54 finished 40000 pairs
2017-11-28 19:22:58 finished 45000 pairs
2017-11-28 19:27:29 finished 50000 pairs
2017-11-28 19:32:07 finished 55000 pairs
2017-11-28 19:38:11 finished 60000 pairs
2017-11-28 19:42:25 finished 65000 pairs
2017-11-28 19:46:37 finished 70000 pairs
2017-11-28 19:51:14 finished 75000 pairs
2017-11-28 19:55:30 finished 80000 pairs
2017-11-28 19:59:56 finished 85000 pairs
2017-11-28 20:05:07 finished 90000 pairs
2017-11-28 20:11:55 finished 95000 pairs
2017-11-28 20:19:23 finished 100000 pairs
2017-11-28 20:24:56 finished 105000 pairs
2017-11-28 20:29:25 finished 110000 pairs
2017-11-28 20:33:55 finished 115000 pairs
2017-11-28 20:38:

In [5]:
X_test=[]
f_test=open('data_format1/test_format1.csv')
cnt=0
print_every=5000
for line in f_test:
    l=line.strip()
    if l=='user_id,merchant_id,prob':
        continue
    [user,seller,_]=l.split(',')
    
    if user not in result_user:
        l1,l2=calc_user_feature(user)
        result_user[user]=(l1,l2)
    else:
        l1,l2=result_user[user]
        
    X_test.append([l1,l2])
    
    if cnt==0 or (cnt+1)%print_every==0 or (cnt+1)==num_test:
        print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())),
              'finished '+str(cnt+1)+' pairs')
    cnt+=1

X_test=np.array(X_test)
print(X_test.shape)
np.save('data/x_bias_test.npy', X_test)

2017-11-28 22:40:36 finished 1 pairs
2017-11-28 22:44:09 finished 5000 pairs
2017-11-28 22:48:28 finished 10000 pairs
2017-11-28 22:52:36 finished 15000 pairs
2017-11-28 22:56:52 finished 20000 pairs
2017-11-28 23:00:51 finished 25000 pairs
2017-11-28 23:04:57 finished 30000 pairs
2017-11-28 23:09:00 finished 35000 pairs
2017-11-28 23:13:00 finished 40000 pairs
2017-11-28 23:16:55 finished 45000 pairs
2017-11-28 23:20:52 finished 50000 pairs
2017-11-28 23:24:42 finished 55000 pairs
2017-11-28 23:28:43 finished 60000 pairs
2017-11-28 23:32:46 finished 65000 pairs
2017-11-28 23:36:47 finished 70000 pairs
2017-11-28 23:40:45 finished 75000 pairs
2017-11-28 23:44:45 finished 80000 pairs
2017-11-28 23:48:41 finished 85000 pairs
2017-11-28 23:51:32 finished 90000 pairs
2017-11-28 23:55:05 finished 95000 pairs
2017-11-28 23:59:02 finished 100000 pairs
2017-11-29 00:03:00 finished 105000 pairs
2017-11-29 00:06:57 finished 110000 pairs
2017-11-29 00:10:58 finished 115000 pairs
2017-11-29 00:14:

In [6]:
cur.close()
conn.close()